In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# !pip uninstall tensorflow
#!pip3 install --upgrade tensorflow-gpu
!pip install --upgrade tensorflow
# !pip install grpcio>=1.24.3 --upgrade
# # !pip install -U keras-tuner
# !pip install tensorflow-addons
!pip install keras-rectified-adam

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.3.0)
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=f2ad7ceaab6cf32d4760b17357f421048b88d6cc910f4a713ac06c59b2f6227d
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam


In [ ]:
from keras_radam import RAdam

In [ ]:
import librosa
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import IPython.display as ipd
import librosa.display
import scipy
import glob
import numpy as np
import math
import warnings
import pickle
from sklearn.utils import shuffle
from keras.backend import clear_session
# Before instantiating a tf.data.Dataset obj & before model creation, call:
clear_session()

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
tf.random.set_seed(999)
np.random.seed(999)

#setting global seeds

In [ ]:
!ls '/content/drive/My Drive/daitan/implementation/largescale/records'

val_0.tfrecords    val_132.tfrecords  val_38.tfrecords	val_6.tfrecords
val_100.tfrecords  val_133.tfrecords  val_39.tfrecords	val_70.tfrecords
val_101.tfrecords  val_134.tfrecords  val_3.tfrecords	val_71.tfrecords
val_102.tfrecords  val_135.tfrecords  val_40.tfrecords	val_72.tfrecords
val_103.tfrecords  val_136.tfrecords  val_41.tfrecords	val_73.tfrecords
val_104.tfrecords  val_137.tfrecords  val_42.tfrecords	val_74.tfrecords
val_105.tfrecords  val_138.tfrecords  val_43.tfrecords	val_75.tfrecords
val_106.tfrecords  val_139.tfrecords  val_44.tfrecords	val_76.tfrecords
val_107.tfrecords  val_13.tfrecords   val_45.tfrecords	val_77.tfrecords
val_108.tfrecords  val_14.tfrecords   val_46.tfrecords	val_78.tfrecords
val_109.tfrecords  val_15.tfrecords   val_47.tfrecords	val_79.tfrecords
val_10.tfrecords   val_16.tfrecords   val_48.tfrecords	val_7.tfrecords
val_110.tfrecords  val_17.tfrecords   val_49.tfrecords	val_80.tfrecords
val_111.tfrecords  val_18.tfrecords   val_4.tfrecords	val_81.tfreco

In [ ]:
train_tfrecords_filenames = glob.glob('/content/drive/My Drive/daitan/implementation/largescale/records/train_*')
np.random.shuffle(train_tfrecords_filenames)
train_tfrecords_filenames = list(train_tfrecords_filenames)
val_tfrecords_filenames = glob.glob('/content/drive/My Drive/daitan/implementation/largescale/records/val_*')

In [ ]:
windowLength = 256
overlap      = round(0.25 * windowLength) # overlap of 75%
ffTLength    = windowLength
inputFs      = 48e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 8
print("windowLength:",windowLength)
print("overlap:",overlap)
print("ffTLength:",ffTLength)
print("inputFs:",inputFs)
print("fs:",fs)
print("numFeatures:",numFeatures)
print("numSegments:",numSegments)

windowLength: 256
overlap: 64
ffTLength: 256
inputFs: 48000.0
fs: 16000.0
numFeatures: 129
numSegments: 8


In [ ]:
def tf_record_parser(record):
    keys_to_features = {
        "noise_stft_phase": tf.io.FixedLenFeature((), tf.string, default_value=""),
        'noise_stft_mag_features': tf.io.FixedLenFeature([], tf.string),
        "clean_stft_magnitude": tf.io.FixedLenFeature((), tf.string)
    }

    features = tf.io.parse_single_example(record, keys_to_features)

    noise_stft_mag_features = tf.io.decode_raw(features['noise_stft_mag_features'], tf.float32)
    clean_stft_magnitude = tf.io.decode_raw(features['clean_stft_magnitude'], tf.float32)
    noise_stft_phase = tf.io.decode_raw(features['noise_stft_phase'], tf.float32)

    # reshape input and annotation images
    noise_stft_mag_features = tf.reshape(noise_stft_mag_features, (129, 8,1), name="noise_stft_mag_features")
    clean_stft_magnitude = tf.reshape(clean_stft_magnitude, (129,1, 1), name="clean_stft_magnitude")
    noise_stft_phase = tf.reshape(noise_stft_phase, (129,), name="noise_stft_phase")
    print(noise_stft_mag_features,clean_stft_magnitude)
    return noise_stft_mag_features,clean_stft_magnitude

Create tf.data.Dataset

In [ ]:
train_dataset = tf.data.TFRecordDataset([train_tfrecords_filenames])
train_dataset = train_dataset.map(tf_record_parser)
#train_dataset = train_dataset.shuffle(8192)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(100)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print(train_dataset.element_spec)

In [ ]:
test_dataset = tf.data.TFRecordDataset([val_tfrecords_filenames])
test_dataset = test_dataset.map(tf_record_parser)
test_dataset = test_dataset.repeat()
test_dataset = test_dataset.batch(100)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset.element_spec

Model Training

In [ ]:
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, Flatten, Dense, Reshape, Conv2DTranspose, BatchNormalization, Activation, Dropout
from tensorflow.keras import Model, Sequential

In [ ]:
def conv_block(x, filters, kernel_size, strides, padding='same', use_bn=True):
  x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(0,0))(x)
  x = Activation('relu')(x)
  if use_bn:
    x = BatchNormalization()(x)
  return x

In [ ]:
def full_pre_activation_block(x, filters, kernel_size, strides, padding='same', use_bn=True):
  shortcut = x
  in_channels = x.shape[-1]

  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)

  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(filters=in_channels, kernel_size=kernel_size, strides=strides, padding='same')(x)

  return shortcut + x

In [ ]:
def build_model(l1_strength,l2_strength):
  inputs = Input(shape=[numFeatures,numSegments,1])
  x = inputs
  #x=Reshape((129,8,1), input_shape=(129,8,1,1))(x)
  # -----
  x = tf.keras.layers.ZeroPadding2D(((4,4), (0,0)))(x)
  x = Conv2D(filters=18, kernel_size=[9,8], strides=[1, 1], padding='valid', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  #x = Dropout(0.5)(x)
  x = BatchNormalization()(x)
 
  skip0 = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
                 kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(skip0)
  x = BatchNormalization()(x)
 
  x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  # -----
  x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  #x = Dropout(0.5)(x)
  x = BatchNormalization()(x)
 
  skip1 = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
                 kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(skip1)
  x = BatchNormalization()(x)
 
  x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  # ----
  x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  #x = Dropout(0.5)(x)
  x = BatchNormalization()(x)
  
  x = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  # ----
  x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  #x = Dropout(0.5)(x)
  x = BatchNormalization()(x)
 
  x = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
             kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  #x = x + skip1
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  # ----
  x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  #x = Dropout(0.5)(x)
  x = BatchNormalization()(x)
 
  x = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
             kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  #x = x + skip0
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l1_l2(l1_strength, l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)
 
  # ----
  x = tf.keras.layers.SpatialDropout2D(0.2)(x)
  x = Conv2D(filters=1, kernel_size=[129,1], strides=[1, 1], padding='same')(x)
 
  model = Model(inputs=inputs, outputs=x)
 
  optimizer = tf.keras.optimizers.Adam(3e-4)
  #optimizer = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=3e-4)
 
  model.compile(optimizer=optimizer, loss='mse', 
                metrics=[tf.keras.metrics.RootMeanSquaredError('rmse')])
  return model

In [ ]:
model = build_model(l1_strength=0,l2_strength=0)
model.summary()
model.save('/content/drive/My Drive/daitan/implementation/largescale/models/models4.h5')

In [ ]:
def l2_norm(vector):
    return np.square(vector)

def SDR(denoised, cleaned, eps=1e-7): # Signal to Distortion Ratio
    a = l2_norm(denoised)
    b = l2_norm(denoised - cleaned)
    a_b = a / b
    return np.mean(10 * np.log10(a_b + eps))

In [ ]:
from tensorflow.keras.models import load_model
saved_model= load_model('/content/drive/My Drive/daitan/implementation/largescale/tempmodels/tempmodels4.h5')

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='batch')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/daitan/implementation/largescale/tempmodels/tempmodels4.h5', 
                                                         monitor='rmse',verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
model.fit(test_dataset,
         steps_per_epoch=150,
         #validation_data=test_dataset,
         epochs=25000,
         callbacks=[tensorboard_callback,checkpoint_callback]
        )

Epoch 1/25000
149/150 [============================>.] - ETA: 0s - loss: 0.7926 - rmse: 0.8903
Epoch 00001: saving model to /content/drive/My Drive/daitan/implementation/largescale/tempmodels/tempmodels4.h5
150/150 [==============================] - 4s 28ms/step - loss: 0.7904 - rmse: 0.8890
Epoch 2/25000
150/150 [==============================] - ETA: 0s - loss: 0.3417 - rmse: 0.5846
Epoch 00002: saving model to /content/drive/My Drive/daitan/implementation/largescale/tempmodels/tempmodels4.h5
150/150 [==============================] - 5s 30ms/step - loss: 0.3417 - rmse: 0.5846
Epoch 3/25000
148/150 [============================>.] - ETA: 0s - loss: 0.2972 - rmse: 0.5451
Epoch 00003: saving model to /content/drive/My Drive/daitan/implementation/largescale/tempmodels/tempmodels4.h5
150/150 [==============================] - 5s 31ms/step - loss: 0.2972 - rmse: 0.5451
Epoch 4/25000
149/150 [============================>.] - ETA: 0s - loss: 0.2796 - rmse: 0.5288
Epoch 00004: saving model 

Testing

In [ ]:
def read_audio(filepath, sample_rate, normalize=True):
    # print(f"Reading: {filepath}").
    audio, sr = librosa.load(filepath, sr=sample_rate)
    if normalize:
      div_fac = 1 / np.max(np.abs(audio)) / 3.0
      audio = audio * div_fac
    return audio, sr
        
def add_noise_to_clean_audio(clean_audio, noise_signal):
    if len(clean_audio) >= len(noise_signal):
        # print("The noisy signal is smaller than the clean audio input. Duplicating the noise.")
        while len(clean_audio) >= len(noise_signal):
            noise_signal = np.append(noise_signal, noise_signal)

    ## Extract a noise segment from a random location in the noise file
    ind = np.random.randint(0, noise_signal.size - clean_audio.size)

    noiseSegment = noise_signal[ind: ind + clean_audio.size]

    speech_power = np.sum(clean_audio ** 2)
    noise_power = np.sum(noiseSegment ** 2)
    noisyAudio = clean_audio + np.sqrt(speech_power / noise_power) * noiseSegment
    return noisyAudio

def play(audio, sample_rate):
    ipd.display(ipd.Audio(data=audio, rate=sample_rate))  # load a local WAV file

In [ ]:
class FeatureExtractor:
    def __init__(self, audio, *, windowLength, overlap, sample_rate):
        self.audio = audio
        self.ffT_length = windowLength
        self.window_length = windowLength
        self.overlap = overlap
        self.sample_rate = sample_rate
        self.window = scipy.signal.hamming(self.window_length, sym=False)

    def get_stft_spectrogram(self):
        return librosa.stft(self.audio, n_fft=self.ffT_length, win_length=self.window_length, hop_length=self.overlap,
                            window=self.window, center=True)

    def get_audio_from_stft_spectrogram(self, stft_features):
        return librosa.istft(stft_features, win_length=self.window_length, hop_length=self.overlap,
                             window=self.window, center=True)

    def get_mel_spectrogram(self):
        return librosa.feature.melspectrogram(self.audio, sr=self.sample_rate, power=2.0, pad_mode='reflect',
                                           n_fft=self.ffT_length, hop_length=self.overlap, center=True)

    def get_audio_from_mel_spectrogram(self, M):
        return librosa.feature.inverse.mel_to_audio(M, sr=self.sample_rate, n_fft=self.ffT_length, hop_length=self.overlap,
                                             win_length=self.window_length, window=self.window,
                                             center=True, pad_mode='reflect', power=2.0, n_iter=32, length=None)

In [ ]:
cleanAudio, sr = read_audio('/content/drive/My Drive/daitan/implementation/smallscale/audio.wav', sample_rate=fs)
print("Min:", np.min(cleanAudio),"Max:",np.max(cleanAudio))
ipd.Audio(data=cleanAudio, rate=sr) # load a local WAV file

In [ ]:
noiseAudio, sr = read_audio('/content/drive/My Drive/daitan/implementation/smallscale/noise/00 - gold fronts-1.wav', sample_rate=fs)
print("Min:", np.min(noiseAudio),"Max:",np.max(noiseAudio))
ipd.Audio(data=noiseAudio, rate=sr) # load a local WAV file

In [ ]:
cleanAudioFeatureExtractor = FeatureExtractor(cleanAudio, windowLength=windowLength, overlap=overlap, sample_rate=sr)
stft_features = cleanAudioFeatureExtractor.get_stft_spectrogram()
stft_features = np.abs(stft_features)
print("Min:", np.min(stft_features),"Max:",np.max(stft_features))

In [ ]:
noisyAudio = add_noise_to_clean_audio(cleanAudio, noiseAudio)
ipd.Audio(data=noisyAudio, rate=fs) # load a local WAV file

In [ ]:
def prepare_input_features(stft_features):
    # Phase Aware Scaling: To avoid extreme differences (more than
    # 45 degree) between the noisy and clean phase, the clean spectral magnitude was encoded as similar to [21]:
    noisySTFT = np.concatenate([stft_features[:,0:numSegments-1], stft_features], axis=1)
    stftSegments = np.zeros((numFeatures, numSegments , noisySTFT.shape[1] - numSegments + 1))

    for index in range(noisySTFT.shape[1] - numSegments + 1):
        stftSegments[:,:,index] = noisySTFT[:,index:index + numSegments]
    return stftSegments

In [ ]:
noiseAudioFeatureExtractor = FeatureExtractor(noisyAudio, windowLength=windowLength, overlap=overlap, sample_rate=sr)
noise_stft_features = noiseAudioFeatureExtractor.get_stft_spectrogram()

# Paper: Besides, spectral phase was not used in the training phase.
# At reconstruction, noisy spectral phase was used instead to
# perform in- verse STFT and recover human speech.
noisyPhase = np.angle(noise_stft_features)
print(noisyPhase.shape)
noise_stft_features = np.abs(noise_stft_features)

mean = np.mean(noise_stft_features)
std = np.std(noise_stft_features)
noise_stft_features = (noise_stft_features - mean) / std

In [ ]:
predictors = prepare_input_features(noise_stft_features)

In [ ]:
predictors = np.reshape(predictors, (predictors.shape[0], predictors.shape[1], 1, predictors.shape[2]))
predictors = np.transpose(predictors, (3, 0, 1, 2)).astype(np.float32)
print('predictors.shape:', predictors.shape)

In [ ]:
STFTFullyConvolutional = saved_model.predict(predictors)
print(STFTFullyConvolutional.shape)

In [ ]:
def revert_features_to_audio(features, phase, cleanMean=None, cleanStd=None):
    # scale the outpus back to the original range
    if cleanMean and cleanStd:
        features = cleanStd * features + cleanMean

    phase = np.transpose(phase, (1, 0))
    features = np.squeeze(features)

    # features = librosa.db_to_power(features)
    features = features * np.exp(1j * phase)  # that fixes the abs() ope previously done

    features = np.transpose(features, (1, 0))
    return noiseAudioFeatureExtractor.get_audio_from_stft_spectrogram(features)

In [ ]:
denoisedAudioFullyConvolutional = revert_features_to_audio(STFTFullyConvolutional, noisyPhase, mean, std)
print("Min:", np.min(denoisedAudioFullyConvolutional),"Max:",np.max(denoisedAudioFullyConvolutional))
ipd.Audio(data=denoisedAudioFullyConvolutional, rate=fs) # load a local WAV file

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharey=True)

ax1.plot(cleanAudio)
ax1.set_title("Clean Audio")

ax2.plot(noisyAudio)
ax2.set_title("Noisy Audio")

ax3.plot(denoisedAudioFullyConvolutional)
ax3.set_title("Denoised Audio")
print(np.sqrt(np.mean((cleanAudio[:-42]-denoisedAudioFullyConvolutional)**2)))
print(np.sqrt(np.mean((noisyAudio[:-42]-denoisedAudioFullyConvolutional)**2)))